In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from isolated_backtester import *
from trade_data import fetch_ohlcvs
from passivbot_isolated_margin import load_settings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)

In [ ]:
symbol = 'XMR/BTC'
n_days = 90

In [ ]:
all_settings = load_settings('default')
settings = all_settings['symbols'][symbol]
settings

In [ ]:
# modify settings

settings['account_equity_pct_per_entry'] = 0.0001
settings['account_equity_pct_per_hour'] = 0.001
settings['ema_spans_minutes'] = [15, 25, 40, 64, 102, 164, 263, 421, 675, 1080]
settings['entry_spread'] = 0.001
settings['entry_vol_modifier_exponent'] = 20
settings['long'] = True
settings['markup_pct_multiplier'] = 0.5
settings['max_leverage'] = 4
settings['max_markup_pct'] = 0.05
settings['min_exit_cost_multiplier'] = 20
settings['min_markup_pct'] = 0.0025
settings['n_days_to_min_markup'] = 12
settings['shrt'] = True

In [ ]:
ohlcvs = fetch_ohlcvs(symbol, n_days)

In [ ]:
df = prep_df(ohlcvs, settings)

In [ ]:
df.avg.plot()

In [ ]:
logs, trades = backtest(df, settings)

In [ ]:
ldf = pd.DataFrame(logs).set_index('timestamp')
tdf = pd.DataFrame(trades).set_index('timestamp')
ldf

In [ ]:
ldf.equity.plot()

In [ ]:
buys = tdf[tdf.side == 'buy']
sels = tdf[tdf.side == 'sel']
lp = ldf.long_exit_price
sp = ldf.shrt_exit_price
le = buys[buys.type == 'entry'] # long entry
lx = sels[sels.type == 'exit']  # long exit
se = sels[sels.type == 'entry'] # shrt entry
sx = buys[buys.type == 'exit']  # shrt exit

In [ ]:
lp.plot()
le.price.plot(style='b.--')
lx.price.plot(style='ro')

In [ ]:
sp.plot()
se.price.plot(style='r.--')
sx.price.plot(style='bo')

In [ ]:
bvwap = buys.cost.sum() / buys.amount.sum()
svwap = sels.cost.sum() / sels.amount.sum()
levwap = le.cost.sum() / le.amount.sum()
lxvwap = lx.cost.sum() / lx.amount.sum()
sevwap = se.cost.sum() / se.amount.sum()
sxvwap = sx.cost.sum() / sx.amount.sum()
print('lxvwap / levwap', lxvwap / levwap)
print('sevwap / sxvwap', sevwap / sxvwap)
print('svwap / bvwap', svwap / bvwap)
print('sevwap / levwap', sevwap / levwap)